# IHLT PROJECT - SEMANTIC TEXTUAL SIMILARITY

Students:
   - Albert Rial
   - Utku Ünal

## INTRODUCTION

In this project we have done a task included in the SemEval (Semantic Evaluation Exercises), a series of workshops which have the main aim of the evaluation and comparision of semantic analysis systems.

The task done has been Semantic Textual Similarity (STS), also known as paraphrases detection. A paraphrase between two sentences or texts is when both have the same meaning using different words. The task consists in given two pair of sentences, provide a similarity value between them.

To implement our system we have done several tasks:   
1) First we have implemented functions needed to load the train and test data and preprocess the sentences.   
2) Then we have defined features between sentences in order to describe its similarity or difference.   
3) Finally we have used a Support Vector Regression in order to train our system with the features defined and predict the similarity of test sentences.   

*Note*: in this project we have tried to avoid the use of pre-trained models and more datasets than the ones given in the competition and used in the subject. We have used the concepts and techniques learnt in class and we think that we have obtained very good results with them. However, in order to increase the accuracy and try to overpass the team in the first position of SemEval 2012, we have done a similar version of this project that, in addition of our features, it also uses a pre-trained model of sentence embeddings called InferSent. Is a model implemented by Facebook that provides semantic representations for English sentences. This system can be found in the other notebook.

In this notebook you will find first, our solution implemented and commented, and in the end the details of the process followed and our conclusions.

## SOLUTION

### IMPORTS

In [1]:
import nltk
import glob
import string
import numpy as np
import math
import torch

from scipy.stats import pearsonr

from nltk import pos_tag
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.stem import WordNetLemmatizer
from nltk.metrics import jaccard_distance
from nltk.probability import FreqDist
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import TrigramCollocationFinder
from nltk.wsd import lesk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.corpus import sentiwordnet
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')

import sklearn
from sklearn.svm import SVR

#nltk.download('maxent_ne_chunker')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('words')
#nltk.download('sentiwordnet')

### LOAD DATA AND PREPROCESS

In this section we define the functions that load the data (sentences and gold-standard) from the files and preprocess them in order to tokenize the words, remove punctuation and stopwords, get the lemmas, etc. These functions are:

**get_gs**: reads the gold-standard scores from a set of files and returns a list with them.

In [2]:
def get_gs(file):
    gs = []
    gs_files = glob.glob(file)
    for name in gs_files:
        with open(name, encoding="utf8") as f:
            for line in f:
                score = line.strip().split('\t')
                gs.append(float(score[0]))
    return gs

**get_sentences**: reads the pairs of sentences from a set of files and returns two lists with the sentences. The first list contains the first sentence of each pair and the second list the second sentence.

In [3]:
def get_sentences(file):
    sentences1 = []
    sentences2 = []
    
    input_files = glob.glob(file)
    
    for name in input_files:
        with open(name, encoding="utf8") as f:
            for line in f:
                # Split
                pair_of_sentences = line.strip().split('\t')

                # Tokenize
                sentences1.append(pair_of_sentences[0])
                sentences2.append(pair_of_sentences[1])

    return sentences1, sentences2

**get_all_words**: given a sentence, returns its words in lower case, without taking into account the following punctuation characters !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~

In [4]:
def get_all_words(sentence):
    return [word.lower() for word in nltk.word_tokenize(sentence) if word not in string.punctuation]

**get_words**: like the previous function, given a sentence, returns its words in lower case. In this case apart from removing the punctuation we also remove the english stopwords (‘the’, ‘is’, ‘are’, ...)

In [5]:
def get_words(sentence):
    return [word.lower() for word in nltk.word_tokenize(sentence) if word not in string.punctuation and word.lower() not in stopwords.words('english')]

**transform_tag**: function that transform a POS tag given by the NLTK POS-tagger to the format that WordNetLemmatizer and Lesk function can undersantd.

In [6]:
def transform_tag(tag):
    if tag[0] in {'N', 'V', 'R'}:
        return tag[0].lower()
    elif tag[0] in {'J'}:
        return 'a'
    else:
        return tag[0]

**get_lemmas**: given a list with pairs of (word, pos_tag) it returns a list with the lemmas of the words

In [7]:
def get_lemmas(pos_tags):
    lemmas = []
    
    wnl = WordNetLemmatizer()
    
    for pos_tag in pos_tags:
        word = pos_tag[0].lower()
        tag = transform_tag(pos_tag[1])
        
        if tag in {'n', 'v', 'r', 'a'}:
            lemmas.append(wnl.lemmatize(word, pos=tag))
        else:
            lemmas.append(word)
    return lemmas

**get_word_importance**: given a list of sentences computes the frequency of each word in all the sentences and returns a dictionary with the importance of each word (considering as importance, the total number of words divided by the frequency of each word). We consider that the words with more meaning are the ones that appear less in the corpus. We use the log in the division in order to have numbers of less magnitude.

In [8]:
def get_word_importance(sentences):
    freq = FreqDist()
    total_freq = 0
    
    
    for sentence in sentences:
        all_words = get_all_words(sentence)
        
        for word in all_words:
            # Sum 1 to the freq of the word 
            freq[word.lower()] += 1
            
            # Sum 1 to the total nb of words
            total_freq += 1
                    
    importance = {}
    for word in freq.keys():
        importance[word] = math.log(float(total_freq) / float(freq[word]))
                    
    return importance

### FEATURES

In this section we define all our features. We have several functions that given two sentences, words or lemmas return a float representing a feature of similarity between them. We have both lexical and syntactic features, however not all of them have finally been used because of its performance.
The features defined are the ones below:

**jaccard_similarity**: given two lists of words the function computes the jaccard similarity between them.

In [9]:
def jaccard_similarity(words1, words2):
    return 1-jaccard_distance(set(words1), set(words2))

**cosine_similarity**: given two sets of words we compute the cosine similarity between them.

In [10]:
def cosine_similarity(words1_set, words2_set):
    l1 =[];l2 =[] 
    rvector = words1_set.union(words2_set)  
    for w in rvector: 
        if w in words1_set: l1.append(1)
        else: l1.append(0) 
        if w in words2_set: l2.append(1) 
        else: l2.append(0) 
    c = 0
    # Cosine formula  
    for i in range(len(rvector)): 
            c+= l1[i]*l2[i] 
    cosine = c / float((sum(l1)*sum(l2))**0.5) 
    return cosine

**synsets_similarity**: in this function first, we receive two lists of lemmas and a method for computing similarity between synsets (path, lch, wup, lin). For each lemma in the first list, we calculate the maximum synset similarity between all the lemmas in the second list. With the maximum similarites obtained we compute the mean similarity between the two lists of lemmas. Moreover, as computing the wordnet similarities between one lemma in respect to other is not the same as doing it  inversely, we compute two means.

As one lemma has more than one synsets, to compute the synset similarity between one lemma and another we use the function *max_similarity_synsets*. This function receives two lemmas, gets the synsets of each one of them, and for each synset of one lemma computes the similarity between the synsets of the other lemma and returns the maximum similarity. To compute the similarity between synsets it uses *wordnet_similarity*, that is a simple function that calls the wordnet similarity functions given two synsets and the specific method to use.   


In [11]:
# Dictionary where we store the similarity between synsets in order to reduce computational cost
computed_synsets_sim = {}

def wordnet_similarity(s1, s2, method):
    if method == "path" and s1 is not None and s2 is not None:
        return s1.path_similarity(s2)
    
    elif method == "lch" and s1 is not None and s2 is not None and s1.pos == s2.pos:
        return s1.lch_similarity(s2)
    
    elif method == "wup" and s1 is not None and s2 is not None:
        return s1.wup_similarity(s2)
    
    elif method == "lin" and s1 is not None and s2 is not None and s1.pos == s2.pos and s1.pos in {'n', 'v', 'r', 'a'}:
        return s1.lin_similarity(s2)
    
    else:
        return None

def max_similarity_synsets(l1, l2, method):
    # If are the same we return the max value
    if l1 == l2:
        if method == "lch":
            return 3
        else:
            return 1
        
    # If we have computed before the similarity we don't compute anything
    elif (l1,l2,method) in computed_synsets_sim:
        return computed_synsets_sim[(l1,l2,method)]
    
    # Get synsets
    synsets1 = wordnet.synsets(l1)
    synsets2 = wordnet.synsets(l2)
    
    similarities = []
    for s1 in synsets1:
        for s2 in synsets2:
            # Get the similarity between synsets
            similarity = wordnet_similarity(s1, s2, method)
            if similarity is not None:
                similarities.append(similarity)
    
    # Return the maximum similarity
    if len(similarities) > 0:
        computed_synsets_sim[(l1,l2,method)] = max(similarities)
        return max(similarities)
    else:
        computed_synsets_sim[(l1,l2,method)] = 0
        return 0

def synsets_similarity(lemmas1, lemmas2, method):
    sum_sim1 = 0
    for l1 in lemmas1:
        sum_sim1 += max([max_similarity_synsets(l1, l2, method) for l2 in lemmas2])
    mean_sim1 = sum_sim1 / len(lemmas1)
    
    sum_sim2 = 0
    for l2 in lemmas2:
        sum_sim2 += max([max_similarity_synsets(l2, l1, method) for l1 in lemmas1])
    mean_sim2 = sum_sim2 / len(lemmas2)
    
    if mean_sim1 > 0 or mean_sim2 > 0:
        return (2 * mean_sim1 * mean_sim2)/(mean_sim1+mean_sim2)
    else:
        return 0

**same_num_entities**: this is a very basic function that, given two lists of name entities, counts for each list the number of entities of an specific label and returns 1 if both have the same number of them, and 0 otherwise.

In [12]:
def same_num_entities(ne1, ne2, entity):
    num1 = 0 
    for p1 in ne1:
        if isinstance(p1, nltk.tree.Tree) and p1.label()==entity:
            num1 += 1
            
    num2 = 0    
    for p2 in ne2:
        if isinstance(p2, nltk.tree.Tree) and p2.label()==entity:
            num2 += 1
        
    if num1 == num2:
        return 1
    else:
        return 0

**sentiment_similarity**: given two lists of lemmas it computes the polarity of each list summing the polarity of each lemma, and then computes the absolute difference between them (it is normalized using the maximum polarity). To calculate the polarity of each lemma uses the function *get_sentiment_score*, which uses the SentiWordnet to get the positive score and negative score of each synset of the lemma, and sum all to get the polarity.

In [13]:
def get_sentiment_score(lemma):
    synsets = wordnet.synsets(lemma)
    score = 0
    for s in synsets:
        senti_synset = sentiwordnet.senti_synset(s.name())
        if senti_synset is not None:
            score += senti_synset.pos_score() - senti_synset.neg_score()
    return score
    
def sentiment_similarity(lemmas1, lemmas2):
    polarity1 = 0
    for l1 in lemmas1:
        polarity1 += get_sentiment_score(l1)
        
    polarity2 = 0
    for l2 in lemmas2:
        polarity2 += get_sentiment_score(l2)
    
    if polarity1 > 0 or polarity2 > 0:
        return abs(polarity1-polarity2) / max(polarity1, polarity2)
    else:
        return 0   

**lesk_similarity**: given two lists of words we apply lesk algorithm to do word sense disambiguation, and we compute the jaccard similarity between the senses obtained.

In [14]:
def lesk_similarity(words1, words2):
    pos_tags1 = pos_tag(words1)
    pos_tags2 = pos_tag(words2)
    
    lesk_synsets1 = []
    for i in range(0, len(words1)):
        if(pos_tags1[i] in {'n', 'v', 'r', 'a'}):
            lesk_synsets1.append(lesk(words1, words1[i], pos_tags1[i]))
            
    lesk_synsets2 = []
    for i in range(0, len(words2)):
        if(pos_tags2[i] in {'n', 'v', 'r', 'a'}):
            lesk_synsets2.append(lesk(words2, words2[i], pos_tags2[i]))
    
    if len(lesk_synsets1) > 0 and len(lesk_synsets2) > 0:
        return 1-jaccard_distance(set(lesk_synsets1), set(lesk_synsets2))
    else:
        return 0

**length_difference**: given two list of words it returns the difference between their lengths (normalized using the maximum length).

In [15]:
def length_difference(words1, words2):
    return abs(len(words1)-len(words2)) / max(len(words1), len(words2))

**unigram_similarity**: given two lists of words it counts the number of same words that we have in both lists.

In [16]:
def unigram_similarity(words1, words2):
    count_same = 0
    for w in words1:
        count_same += min(words1.count(w), words2.count(w))
    
    if len(words1) > 0 or len(words2) > 0:
        return 2*count_same/(len(words1)+len(words2))
    else:
        return 0

**unigram_similarity_importance**: given two lists of words it counts the number of same words that we have in both lists but when counting a word we take into account its importance in the corpus. This important is computed using previous function *get_word_importance* and stored in the dictionary *word_importance*. The less a word appears in the corpus the more important we consider it, and vice versa.

In [17]:
def unigram_similarity_importance(words1, words2):
    count_same = 0
    for w in words1:
        # Count number of same words
        count_same += min(words1.count(w), words2.count(w)) * word_importance.get(w, max_importance)
        
    if len(words1) > 0 or len(words2) > 0:
        return 2*count_same/(len(words1)+len(words2))
    else:
        return 0

**bigram_similarity**: given two lists of words this function counts the number of same bigrams that we have in both lists.

In [18]:
def bigram_similarity(words1, words2):
    finder1 = BigramCollocationFinder.from_words(words1)
    finder2 = BigramCollocationFinder.from_words(words2)
    
    # We get the bigrams of first sentence and its frequency
    bigrams1 = []
    freq1 = []
    for b1 in finder1.ngram_fd.items():
        bigrams1.append(b1[0])
        freq1.append(b1[1])
    
    # We get the bigrams of second sentence and its frequency
    bigrams2 = []
    freq2 = []
    for b2 in finder2.ngram_fd.items():
        bigrams2.append(b2[0])
        freq2.append(b2[1])
    
    count = 0
    for i in range(len(bigrams1)):
        if bigrams1[i] in bigrams2:
            # Count number of same bigrams
            count += min(freq1[i], freq2[bigrams2.index(bigrams1[i])])
            
    if len(words1) > 0 or len(words2) > 0:
        return 2*count/(len(words1)+len(words2))
    else:
        return 0

**trigram_similarity**: given two lists of words this function counts the number of same trigrams that we have in both lists.

In [19]:
def trigram_similarity(words1, words2):
    finder1 = TrigramCollocationFinder.from_words(words1)
    finder2 = TrigramCollocationFinder.from_words(words2)
    
    # We get the bigrams of first sentence and its frequency
    trigrams1 = []
    freq1 = []
    for t1 in finder1.ngram_fd.items():
        trigrams1.append(t1[0])
        freq1.append(t1[1])
    
    # We get the trigrams of second sentence and its frequency
    trigrams2 = []
    freq2 = []
    for t2 in finder2.ngram_fd.items():
        trigrams2.append(t2[0])
        freq2.append(t2[1])
    
    count = 0
    for i in range(len(trigrams1)):
        if trigrams1[i] in trigrams2:
            # Count number of same trigrams
            count += min(freq1[i], freq2[trigrams2.index(trigrams1[i])])
            
    if len(words1) > 0 or len(words2) > 0:
        return 2*count/(len(words1)+len(words2))
    else:
        return 0

#### Call previous functions to preprocess sentences and compute features
In the following function we receive two sentences and using the first functions defined in this notebook we preprocess them, in order to get all words, words without stopwords, POS tags, lemmas and NEs. Once preprocessed, we compute the features between each pair of sentences and we append the features to our features array.

In [25]:
def get_features(sentences1, sentences2):
    features = []
    for i in range(len(sentences1)):
        sentence1 = sentences1[i]
        sentence2 = sentences2[i]
        
        # Get all words
        all_words1 = get_all_words(sentence1)
        all_words2 = get_all_words(sentence2)
        
        # Get words without stopwords
        words1 = get_words(sentence1)
        words2 = get_words(sentence2)
        
        # POS tags
        pos_tags1 = pos_tag(words1)
        pos_tags2 = pos_tag(words2)

        # Lemmas
        lemmas1 = get_lemmas(pos_tags1)
        lemmas2 = get_lemmas(pos_tags2)

        # Name entities
        ne1 = ne_chunk(pos_tags1)
        ne2 = ne_chunk(pos_tags2)
        
        # Features
        features.append([jaccard_similarity(lemmas1, lemmas2),
                         jaccard_similarity(words1, words2),
                         jaccard_similarity(all_words1, all_words2),
                         #cosine_similarity(set(words1),set(words2)),
                         #cosine_similarity(set(all_words1),set(all_words2)),
                         #cosine_similarity(set(lemmas1),set(lemmas2)),
                         synsets_similarity(lemmas1, lemmas2, "lch"),
                         synsets_similarity(lemmas1, lemmas2, "path"),
                         synsets_similarity(lemmas1, lemmas2, "wup"),
                         synsets_similarity(lemmas1, lemmas2, "lin"),
                         #same_num_entities(ne1, ne2, "PERSON"),
                         #same_num_entities(ne1, ne2, "ORGANIZATION"),
                         #same_num_entities(ne1, ne2, "LOCATION"),
                         #same_num_entities(ne1, ne2, "GPE"),
                         #same_num_entities(ne1, ne2, "FACILITY"),
                         #lesk_similarity(lemmas1, lemmas2),
                         #sentiment_similarity(lemmas1, lemmas2),
                         length_difference(all_words1, all_words2),
                         length_difference(lemmas1, lemmas2),
                         unigram_similarity(lemmas1, lemmas2),
                         unigram_similarity(all_words1, all_words2),
                         unigram_similarity(words1, words2),
                         unigram_similarity_importance(words1, words2),
                         unigram_similarity_importance(lemmas1, lemmas2),
                         unigram_similarity_importance(all_words1, all_words2),
                         bigram_similarity(words1, words2),
                         bigram_similarity(lemmas1, lemmas2),
                         bigram_similarity(all_words1, all_words2),
                         trigram_similarity(words1, words2),
                         trigram_similarity(lemmas1, lemmas2),
                         trigram_similarity(all_words1, all_words2)
                        ])
    return features    

### MAIN

Here we have the main code of the notebook where we call all the previous functions.

#### COMPUTE TRAIN AND TEST FEATURES

Fists we load the train and test sentences, the word importance dictionary and we compute the features of train and test. We also scale them before training.

In [32]:
# Get train sentences
train_sentences1, train_sentences2 = get_sentences('train/STS.input.*')

# Get importance of each word in train corpus
word_importance = get_word_importance(train_sentences1 + train_sentences2)
max_importance = max(word_importance.values())
min_importance= min(word_importance.values())

# Get train features and gs
features_train = get_features(train_sentences1, train_sentences2)
gs_train = get_gs('train/STS.gs.*')

# Scale train features
scaler = sklearn.preprocessing.StandardScaler();
scaler.fit(features_train);
features_train_scaled = scaler.transform(features_train)

# Get test sentences, features and gs
test_sentences1, test_sentences2 = get_sentences('test-gold/STS.input.*')
features_test = get_features(test_sentences1, test_sentences2)  
gs_test = get_gs('test-gold/STS.gs.*')

# Scale test features
features_test_scaled = scaler.transform(features_test)

#### TRAIN AND PREDICT

Next, after computing and scaling the features we train our model and predict with test sentences.

In [33]:
# Train SVR
svr = SVR(kernel = 'rbf', gamma = 0.01, C = 100, epsilon = 0.75, tol = 1)
svr.fit(features_train_scaled, gs_train)

# Predict
test_predict = svr.predict(features_test_scaled)

## RESULT

Finally, we obtain the pearson correlation with the predicted results of our system and the gold-standard.

In [34]:
correlation = pearsonr(test_predict, gs_test)[0]
print("Pearson correlation:", correlation)

Pearson correlation: 0.7801798091320731


## PROCESS AND CONCLUSIONS

### PROCESS

First we implemented the functions needed to load the train and test data and preprocess the sentences. 

After that we started defining features between sentences. At the beginning we had very simple ones:   
- Jaccard similarity between lemmas and words.
- Cosine similarity between lemmas and words.
- LCH similarity between synsets.

With three features defined, we started thinking about what we could use in order to train our system. First, we thought about using a CNN (Convolutional Neural Network) but after analyzing the train dataset we saw that maybe it was too small to use that kind of model. For that reason, we decided to implement a linear regression using Support Vector Regression (sklearn.svm.SVR).

With those three features and a linear regression we obtained a pearson correlation of **61.8%** between gold-standard and our results.

The next features defined were:
- Count if sentences had the same number of entities of the same kind (PERSON, ORGANIZATION, LOCATION, ...)
- Length difference
- Sentiment polarity similarity
- Lesk synsets similarity

When adding these features we saw an improvement and we obtained a pearson correlation of **63.2%**. Then we evaluated which features were the ones that were more important in our dataset and we saw that the most important were the Jaccard similarity between Lemmas and the LCH similarity between Synsets. For that reason we decided to try to implement similarity between Synsets but using another method:
- Path similarity between synsets.

In this point, we did not only add these feature. We also started trying different parameters of the Support Vector Regression model. We obtained the best values when using the *rbf* kernel instead of *linear*, so we changed it. We also changed the kernel coefficient (gamma), the regularization parameter (C), the epsilon (epsilon) and the tolerance for stopping criterion (tol).

Changing those values and with the new feature added, we obtained a higher pearson correlation: **66.4%**.

After that, we analyzed in which type of sentences we were failing most and we saw that we were failing in sentences that shared some trigrams and bigrams. For that reason we implemented the following features:
- Unigram similarity (number of same unigrams/words that both sentences share)
- Bigram similarity (number of same bigrams that both sentences share)
- Trigram similarity (number of same trigrams that both sentences share)

Using those features we obtained a pearson correlation of **68.2%**. However, we thought that when comparing words in the unigram similarity feature, we could take into account how significant is the word we are comparing. We consider the approach that a word is more significant than other if it appears less in the corpus. For that reason, we computed the importance of each word in the training corpus as the total number of words divided by the frequency of that word. This importance is used in the following feature:
- Unigram similarity giving more weight to the important unigrams/words.

With this feature we obtained a higher pearson correlation: **70.6%**. However, at this point we tested again different parameters and obtained a pearson correlation of **73.6%**.

Finally to improve that pearson correlation we tried to use more similarity methods between Synsets. We did the ones left:
- Lin similarity between synsets
- Wup similarity between synsets

With these last features we obtained a pearson correlation of **75.7%**, overpassing the mark of the participant at position 10th in SemEval 2012.

After getting this result we thought that we could try to remove some features and try different combinations in order to obtain even a better result. In fact, analyzing the importance of each feature by removing them one by one, we saw that there were some features we could remove, as we were obtaining better results without them. These features are:
- Cosine similarity between lemmas and words.
- Count if sentences had the same number of entities of the same kind (PERSON, ORGANIZATION, LOCATION, ...)
- Length difference
- Sentiment polarity similarity
- Lesk synsets similarity

Removing those features and tuning the parameters of the SVR we finally obtained a pearson of: **78.02%**.


### CONCLUSIONS
The best result obtained without using the pre-trained model InferSent is **78.02%**. We think it is a very good result as we would overpass the 8th position in the SemEval competition and we would be close to the result obtained of the 5th participant. In addition, we have to take into account that we use very basic techniques and without more data than the one provided by the competition. However, a disadvantage of our solution is that the computational cost is quite high. This is because of the features that compute the similarity between synsets, where we have a 4-inner loop when comparing all synsets of all lemmas between the two sentences. We have tried to reduce the computational cost by comparing less synsets, but the resulst obtained were worst. For that reason, and as in this competition there is no efficiency requirement, we have done the comparision with all synsets.

In the other notebook you can see the result obtained when using InferSent with some of our features. In that notebook we have included some comments of that result. We advance that with only using InferSent, you get worst results. Nevertheless, when using InferSent combined with our system we have been able to obtain a pearson correlation of **82.46%**, which overpasses the result of the winner of the SemEval 2012 competition.

Personally, we think that this has been a very good project for us as we have been able to apply most of the concepts and techniques learnt in the subject. In fact, only using things seen and made in the subject we have been able to get a very good result.